# Example Led 3D Interactive[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kennedym-ds/semiconductor_sim/blob/main/examples/example_led_3d_interactive.ipynb)[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/kennedym-ds/semiconductor_sim/main?filepath=examples/example_led_3d_interactive.ipynb)---*Interactive semiconductor device simulation using SemiconductorSim*

In [ ]:
# Setup headless-safe plotting (Matplotlib)
from semiconductor_sim.utils.plotting import use_headless_backend, apply_basic_style
use_headless_backend('Agg')
apply_basic_style()


In [1]:
# examples/interactive_led_3d.ipynb

import numpy as np
from semiconductor_sim import LED
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Define interactive widgets
doping_p_slider = widgets.FloatLogSlider(
    value=1e17,
    base=10,
    min=14,
    max=20,
    step=0.1,
    description='Doping p:',
    disabled=False,
    continuous_update=False,
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

doping_n_slider = widgets.FloatLogSlider(
    value=1e17,
    base=10,
    min=14,
    max=20,
    step=0.1,
    description='Doping n:',
    disabled=False,
    continuous_update=False,
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

efficiency_slider = widgets.FloatSlider(
    value=0.2,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Efficiency:',
    disabled=False,
    continuous_update=False,
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

temperature_slider = widgets.IntSlider(
    value=300,
    min=200,
    max=400,
    step=10,
    description='Temperature (K):',
    disabled=False,
    continuous_update=False,
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

def update_3d_plot(doping_p, doping_n, efficiency, temperature):
    # Define voltage range
    voltage_iv = np.linspace(-0.5, 0.7, 50)
    temperature_range = np.linspace(200, 400, 50)
    V, T = np.meshgrid(voltage_iv, temperature_range)

    # Initialize the LED
    led = LED(doping_p=doping_p, doping_n=doping_n, efficiency=efficiency, temperature=temperature, B=1e-10)

    # Assume constant carrier concentrations
    n_conc = 1e16  # cm^-3
    p_conc = 1e16  # cm^-3

    # Calculate current over the meshgrid
    V_flat = V.flatten()
    T_flat = T.flatten()
    led.temperature = T_flat  # Update temperature for each point
    current, emission, recombination = led.iv_characteristic(V_flat, n_conc, p_conc)
    current = current.reshape(V.shape)

    # Create 3D surface plot
    fig = go.Figure(data=[go.Surface(z=current, x=voltage_iv, y=temperature_range, colorscale='Viridis')])

    fig.update_layout(title='LED Current vs. Voltage and Temperature',
                      scene=dict(
                          xaxis_title='Voltage (V)',
                          yaxis_title='Temperature (K)',
                          zaxis_title='Current (A)'
                      ),
                      autosize=False,
                      width=800,
                      height=800)

    fig.show()

# Create interactive widget
interactive_3d_plot = widgets.interactive(update_3d_plot,
                                          doping_p=doping_p_slider,
                                          doping_n=doping_n_slider,
                                          efficiency=efficiency_slider,
                                          temperature=temperature_slider)

# Display the widgets and plot
display(interactive_3d_plot)


interactive(children=(FloatLogSlider(value=1e+17, continuous_update=False, description='Doping p:', layout=Lay…